## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
# Extracting the Training set

from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
# Extracting the Test set

from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
# Extracting the Sample Submission set

from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
# Extracting the Sample Submission set

from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [8]:
import pandas as pd

lab= pd.read_csv('labels.csv')
lab

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


### Print the count of each category of Dogs given in the dataset



In [8]:
# Count of each category of Dogs in the dataset
lab['breed'].value_counts()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
komondor                 67
brabancon_griffon        67
eskimo_dog               66
briard                   66
Name: breed, Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
# Encode the different labels for Breed class
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder() 
y_encode = enc.fit_transform(lab['breed'].values)

In [12]:
# As can be seen below, 120 different breeds are labelled with values from 0 to 119
print (max(y_encode),  min(y_encode))

119 0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [62]:
# Train set

from tqdm import tqdm
import cv2

X_train=[]
Y_train=[]
img_rows= 128
img_cols = 128

for f, img in tqdm(lab.values):
  train_img= cv2.imread('./train/{}.jpg'.format(f),1)
  train_img_resize = cv2.resize(train_img, (img_rows, img_cols))
  X_train.append(train_img_resize)
  Y_train.append(img)




100%|██████████| 10222/10222 [00:30<00:00, 334.14it/s]


In [12]:
X_train[0]

array([[[ 69, 116, 173],
        [ 38,  91, 136],
        [102, 166, 204],
        ...,
        [130, 204, 237],
        [ 85, 193, 220],
        [ 59, 138, 170]],

       [[ 42,  77, 118],
        [142, 180, 216],
        [119, 162, 197],
        ...,
        [ 80, 152, 194],
        [101, 183, 218],
        [ 74, 177, 212]],

       [[ 65, 107, 151],
        [ 57, 103, 147],
        [ 66, 107, 153],
        ...,
        [ 37, 121, 172],
        [ 73, 145, 183],
        [ 64, 168, 207]],

       ...,

       [[ 77, 119, 190],
        [ 66, 121, 186],
        [ 66, 122, 183],
        ...,
        [ 70,  91, 159],
        [ 59,  89, 154],
        [ 91, 132, 184]],

       [[ 65, 118, 185],
        [ 72, 124, 190],
        [ 74, 125, 187],
        ...,
        [ 54,  63, 132],
        [ 74, 116, 169],
        [121, 169, 223]],

       [[ 63, 116, 183],
        [ 73, 125, 191],
        [ 76, 127, 190],
        ...,
        [ 50,  51, 127],
        [ 31,  62, 128],
        [ 90, 145, 198]]

In [13]:
Y_train[0]

'boston_bull'

In [19]:
X_train.shape

(10222, 128, 128, 3)

In [20]:
Y_train.shape

(10222, 120)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
# Preprocessing steps

# Encoding the class variable

import pandas as pd
Y_train = pd.get_dummies(Y_train)

# Converting X and Y sets to Arrays
import numpy as np
X_train = np.array(X_train)
Y_train = np.array(Y_train)


# Reshaping the X set into 4 dimensions
# convert from int to float
X_train = X_train.reshape(X_train.shape[0],128,128,3).astype('float32')

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
# Splitting the given data into x_train_data and y_train_data set in the ratio of 7:3

from sklearn.model_selection import train_test_split
x_train_data, x_val, y_train_data, y_val = train_test_split(X_train, Y_train, test_size=0.3, random_state=2) 

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [17]:
# Read the data from samples_submission.csv

test= pd.read_csv('sample_submission.csv')
test_img= test['id']
test_img

0        000621fb3cbb32d8935728e48679680e
1        00102ee9d8eb90812350685311fe5890
2        0012a730dfa437f5f3613fb75efcd4ce
3        001510bc8570bbeee98c8d80c8a95ec1
4        001a5f3114548acdefa3d4da05474c2e
                       ...               
10352    ffeda8623d4eee33c6d1156a2ecbfcf8
10353    fff1ec9e6e413275984966f745a313b0
10354    fff74b59b758bbbf13a5793182a9bbe4
10355    fff7d50d848e8014ac1e9172dc6762a3
10356    fffbff22c1f51e3dc80c4bf04089545b
Name: id, Length: 10357, dtype: object

Run the below code to load the test image files in x_test_feature

In [18]:
# Extracting the Test feature from the given file

x_test_feature = []
i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 1)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:32<00:00, 317.81it/s]


In [32]:
x_test_feature[0].shape

(128, 128, 3)

In [65]:
# Printing the shapes of train and validation sets


# convert from int to float
x_train_data = x_train_data.astype('float32')
x_val = x_val.astype('float32')

#Normalizing the input
x_train_data /= 255
x_val /= 255

print (x_train_data.shape)
print (x_val.shape)
print (y_train_data.shape)
print (y_val.shape)

(7155, 128, 128, 3)
(3067, 128, 128, 3)
(7155, 120)
(3067, 120)


Normalize the test data and convert it into 4 dimensions

In [0]:
# Normalizing the data

import numpy as np
x_test_feature= np.array(x_test_feature)

x_test_feature = x_test_feature.reshape(x_test_feature.shape[0],128,128,3)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [14]:
# Basic CNN Network

import numpy as np
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import adam

# Initialize and Build the Model


TRAIN = False
BATCH_SIZE = 32
EPOCHS = 10

# Define the Type of Model
model1 = Sequential()

# 1st Conv Layer with Kernel size 5
model1.add(BatchNormalization(input_shape = (128,128,3)))
model1.add(Convolution2D(32, kernel_size=(5, 5), activation ='relu', input_shape = (128, 128, 3)))
model1.add(Activation('relu'))

# 2nd Conv Layer with Kernel size 3
model1.add(Convolution2D(32, 3, 3))
model1.add(Activation('relu'))

# Fully Connected Dense Layer
model1.add(Flatten())
model1.add(Dense(256))
model1.add(Activation('relu'))

# Final Dense Layer
model1.add(Dense(120))
model1.add(Activation('softmax'))

# Loss and Optimizer
from keras.optimizers import Adam, SGD
opt=SGD(lr=0.001)
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
# Train the model
model1.fit(x_train_data, y_train_data, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, 
              validation_data=(x_val, y_val))
    




Using TensorFlow backend.















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.




Train on 5600 samples, validate on 2400 samples
Epoch 1/10
5600/5600 [==============================] - 14s 3ms/step - loss: 4.7711 - acc: 0.0170 - val_loss: 4.7305 - val_acc: 0.0229
Epoch 2/10
5600/5600 [==============================] - 7s 1ms/step - loss: 4.6436 - acc: 0.0277 - val_loss: 4.6641 - val_acc: 0.0271
Epoch 3/10
5600/5600 [==============================] - 7s 1ms/step - loss: 4.4891 - acc: 0.0527 - val_loss: 4.6473 - val_acc: 0.0213
Epoch 4/10
5600/5600 [==============================] - 7s 1ms/step - loss: 4.2966 - acc: 0.0771 - val_loss: 4.6483 - val_acc: 0.0271
Epoch 5/10
5600/5600 [==============================] - 7s 1ms/step - loss: 4.0721 - acc: 0.1118 - val_loss: 4.6531 - val_acc: 0.0308
Epoch 6/10
5600/5600 [==============================] - 7s 1ms/step - loss: 3.7833 - acc: 0.1573 - val_loss: 4.7972 - val_acc: 0.0329
Epoch 7/10
5600/5600 [==============================] - 7s 1ms/step - loss: 3.4552 - acc: 0.2304 - val_loss: 4.9714 - val_acc: 0.0333
Epoch 8/10


In [53]:
# Basic CNN Network with Batch size 128

# Initialize and Build the Model


TRAIN = False
BATCH_SIZE = 128
EPOCHS = 10

# Define the Type of Model
model1 = Sequential()

# 1st Conv Layer with Kernel size 5
model1.add(BatchNormalization(input_shape = (128,128,3)))
model1.add(Convolution2D(32, kernel_size=(5, 5), activation ='relu', input_shape = (128, 128, 3)))
model1.add(Activation('relu'))

# 2nd Conv Layer with Kernel size 3
model1.add(Convolution2D(32, 3, 3))
model1.add(Activation('relu'))

# Fully Connected Dense Layer
model1.add(Flatten())
model1.add(Dense(256))
model1.add(Activation('relu'))

# Final Dense Layer
model1.add(Dense(120))
model1.add(Activation('softmax'))

# Loss and Optimizer
opt=SGD(lr=0.001)
model1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
# Train the model
model1.fit(x_train_data, y_train_data, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, 
              validation_data=(x_val, y_val))
    




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 7155 samples, validate on 3067 samples
Epoch 1/10
7155/7155 [==============================] - 15s 2ms/step - loss: 4.7647 - acc: 0.0152 - val_loss: 4.7448 - val_acc: 0.0192
Epoch 2/10
7155/7155 [==============================] - 14s 2ms/step - loss: 4.6379 - acc: 0.0294 - val_loss: 4.6761 - val_acc: 0.0280
Epoch 3/10
7155/7155 [==============================] - 14s 2ms/step - loss: 4.4677 - acc: 0.0506 - val_loss: 4.6364 - val_acc: 0.0293
Epoch 4/10
7155/7155 [==============================] - 14s 2ms/step - loss: 4.2253 - acc: 0.0837 - val_loss: 4.5919 - val_acc: 0.0372
Epoch 5/10
7155/7155 [==============================] - 14s 2ms/step - loss: 3.9239 - acc: 0.1276 - val_loss: 4.6296 - val_acc: 0.0408
Epoch 6/10
7155/7155 [==============================] - 14s 2ms/step - loss: 3.4659 - acc: 0.2117 - val_loss: 4.7879 - val_acc: 0.0349
Epoch 7/10
7155/7155 [==============================] - 14s 2ms/step - loss: 2.8654 - acc: 0.3420 - val_loss: 5.0702 - val_acc: 0.0411
Epoch 8

### Use batch_size = 128 and epochs = 10 and execute the model

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
# Improving the model through Data Augmentation
# set data augmentation configuration

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        featurewise_center=False, samplewise_center=False,    featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  zca_whitening=False,  rotation_range=50,  
        width_shift_range=0.1,  height_shift_range=0.1, horizontal_flip=True, 
        vertical_flip=False,shear_range=0.2,zoom_range=0.25,fill_mode='nearest')

val_datagen = ImageDataGenerator(featurewise_center=False, samplewise_center=False,    featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  zca_whitening=False,  rotation_range=50,  
        width_shift_range=0.1,  height_shift_range=0.1, horizontal_flip=True, 
        vertical_flip=False,shear_range=0.2,zoom_range=0.25,fill_mode='nearest')


### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
# Train Generator
train_generator = train_datagen.flow(x_train_data, y_train_data, batch_size=128)

# Validation generator

val_generator = val_datagen.flow(x_val, y_val, batch_size=128)
        

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [24]:
step_size_train=train_generator.n//train_generator.batch_size
model1.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10,validation_data=val_generator)

Epoch 1/10
16/16 [==============================] - 12s 745ms/step - loss: 4.7634 - acc: 0.0147 - val_loss: 8.7239 - val_acc: 0.0089
Epoch 2/10
16/16 [==============================] - 11s 673ms/step - loss: 4.7613 - acc: 0.0198 - val_loss: 8.4155 - val_acc: 0.0089
Epoch 3/10
16/16 [==============================] - 11s 673ms/step - loss: 4.7616 - acc: 0.0191 - val_loss: 8.1303 - val_acc: 0.0089
Epoch 4/10
16/16 [==============================] - 11s 670ms/step - loss: 4.7582 - acc: 0.0191 - val_loss: 7.8370 - val_acc: 0.0089
Epoch 5/10
16/16 [==============================] - 11s 683ms/step - loss: 4.7531 - acc: 0.0198 - val_loss: 7.4631 - val_acc: 0.0089
Epoch 6/10
16/16 [==============================] - 11s 667ms/step - loss: 4.7515 - acc: 0.0219 - val_loss: 7.3098 - val_acc: 0.0089
Epoch 7/10
16/16 [==============================] - 11s 689ms/step - loss: 4.7517 - acc: 0.0225 - val_loss: 7.0228 - val_acc: 0.0089
Epoch 8/10
16/16 [==============================] - 10s 656ms/step - 

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input

path='/content/drive/My Drive/'
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

In [0]:
for layer in base_model.layers:
    layer.trainable = False

Print the summary of the base_model

In [26]:
# Summary of the base model

base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
from keras.layers import GlobalAveragePooling2D
x=base_model.output


In [0]:
x=Flatten()(x) #Flatten
x=Dense(1024,activation='relu')(x) #dense layer 1
x=Dense(256,activation='relu')(x) #dense layer 2
predictions=Dense(120,activation='softmax')(x) #final dense layer with softmax activation


In [0]:
from keras.models import Model
model=Model(inputs=base_model.input,outputs=predictions)

### Make all the layers in the base_model (VGG16) to be non-trainable

In [32]:
# Understanding the layers to be trained and non-trained
for i,layer in enumerate(model.layers):
  print(i,layer.name)

0 input_2
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_2
20 dense_6
21 dense_7
22 dense_8


In [0]:
# Base models to be non-trainable and others to be trainable

for layer in model.layers[:19]:
    layer.trainable=False
for layer in model.layers[19:]:
    layer.trainable=True

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [61]:
# Applying the model on the train and validation set
from keras.optimizers import RMSprop
opt=Adam(lr=0.01)
#opt=RMSprop(lr=0.01, rho=0.9)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=10,validation_data=val_generator)


Epoch 1/10
43/43 [==============================] - 28s 657ms/step - loss: 13.8050 - acc: 0.1369 - val_loss: 14.0985 - val_acc: 0.1154
Epoch 2/10
43/43 [==============================] - 28s 643ms/step - loss: 13.4741 - acc: 0.1562 - val_loss: 13.9132 - val_acc: 0.1296
Epoch 3/10
43/43 [==============================] - 27s 637ms/step - loss: 13.0743 - acc: 0.1777 - val_loss: 13.6686 - val_acc: 0.1425
Epoch 4/10
43/43 [==============================] - 28s 641ms/step - loss: 12.8952 - acc: 0.1894 - val_loss: 13.6136 - val_acc: 0.1475
Epoch 5/10
43/43 [==============================] - 27s 636ms/step - loss: 12.7590 - acc: 0.1993 - val_loss: 13.4249 - val_acc: 0.1583
Epoch 6/10
43/43 [==============================] - 27s 636ms/step - loss: 12.5520 - acc: 0.2109 - val_loss: 13.5291 - val_acc: 0.1529
Epoch 7/10
43/43 [==============================] - 28s 641ms/step - loss: 12.3615 - acc: 0.2231 - val_loss: 13.4153 - val_acc: 0.1621
Epoch 8/10
43/43 [==============================] - 27s

In [66]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   